### Face and Eye Recognition
Before we discuss redaction, lets cover the basics of identifying a face within an image.

Within OpenCV-Python, we can use the cv2.rectangle method to draw a rectangle over an image. We need five parameters for this function: image, start_point, end_point, color, and thickness. 

We can get the start and end points by running a classier to identify the area that contains the face. The code block below uses the haarcascade classifier.  Images must be converted to grayscale before applying the classifier.  The end result is the detectmultiscale function returns an array that correspond to the coordinates for the rectangle that you will input in the opencv rectangle function.

A similar pattern is performed to identify any other features such as the eyes which is done in the code block below. We draw the rectangles for the face and eye over the original image and display it in a widget. 


In [1]:
#https://www.geeksforgeeks.org/how-to-blur-faces-in-images-using-opencv-in-python/
#https://docs.opencv.org/3.1.0/d7/d8b/tutorial_py_face_detection.html#gsc.tab=0
# Importing libraries
import numpy as np
import cv2
import ipywidgets
from IPython.display import display

# A function for plotting the images

oimage_widget = ipywidgets.Image(format='jpeg')
image_widget = ipywidgets.Image(format='jpeg')
# Reading an image using OpenCV
# OpenCV reads images by default in BGR format
image = cv2.imread('lena_color.tif')

# Converting BGR image into a RGB image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# plotting the original image
oimage = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
oimage_widget.value = bytes(cv2.imencode('.jpg', oimage)[1])
display(oimage_widget)

#need to ensure this matches the actual file locations!
fpath='./cascades/haarcascade_frontalface_alt.xml'
epath='./cascades/haarcascade_eye.xml'

face_detect = cv2.CascadeClassifier(fpath)
eye_detect = cv2.CascadeClassifier(epath)

#returns an array that identifies the four points that contain the face
faces = face_detect.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    eyes = eye_detect.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
       

# Display the output
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image_widget.value = bytes(cv2.imencode('.jpg', image)[1])
display(image_widget)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Redact an image - CPU
Now lets blur the are within the rectangle to redact the face using the OpenCV GaussianBlur function. We will not identify eyes in this code block, just the face.

In [2]:
%%time
import numpy as np
import cv2
import ipywidgets
from IPython.display import display
import numpy as np

# Reading an image using OpenCV
image = cv2.imread('lena_color.tif')
image_widget = ipywidgets.Image(format='jpeg')

# Converting BGR image into a RGB image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

fpath='./cascades/haarcascade_frontalface_alt.xml'

face_detect = cv2.CascadeClassifier(fpath)

faces = face_detect.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    roi = cv2.GaussianBlur(roi_color, (23, 23), 30)
    # impose this blurred image on original image to get final image
    image[y:y+roi.shape[0], x:x+roi.shape[1]] = roi

# Display the output
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image_widget.value = bytes(cv2.imencode('.jpg', image)[1])
display(image_widget)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

CPU times: user 200 ms, sys: 28 ms, total: 228 ms
Wall time: 426 ms


### Redact an image -GPU
Now lets use the GPU to run the classifier. 
In this code block, we upload the image to the GPU, run it against the CUDA version of the classifier, the download the array that contains the coodinates and the image to the CPU.  We must download the results to the CPU in order to draw the rectangle and display the image.

In [4]:
%%time
import numpy as np
import cv2
import ipywidgets
from IPython.display import display
import numpy as np
import timeit

image_widget = ipywidgets.Image(format='jpeg')

image = cv2.imread('lena_color.tif')

# Converting BGR image into a RGB image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

fpath='./cascades_cuda/haarcascade_frontalface_alt.xml'

cuImage = cv2.cuda_GpuMat()

cuImage.upload(image)

gray = cv2.cuda.cvtColor(cuImage,cv2.COLOR_BGR2GRAY)

face_detect = cv2.cuda_CascadeClassifier.create(fpath)

faces = face_detect.detectMultiScale(gray).download()
gray = gray.download()

if faces is not None:
    #print(faces[0])

    #this must be done on CPU
    for (x,y,w,h) in faces[0]:
        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]
        roi = cv2.GaussianBlur(roi_color, (23, 23), 30)
        # impose this blurred image on original image to get final image
        image[y:y+roi.shape[0], x:x+roi.shape[1]] = roi

cuImage.upload(image)
fimage = cv2.cuda.cvtColor(cuImage, cv2.COLOR_RGB2BGR)
fimage = fimage.download()
# Display the output

image_widget.value = bytes(cv2.imencode('.jpg', fimage)[1])

display(image_widget)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

CPU times: user 192 ms, sys: 28 ms, total: 220 ms
Wall time: 476 ms


In [6]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}